In [3]:
from splinter import Browser
import re
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
from datetime import datetime
import pandas as pd
import numpy as np
import re
from pathlib import Path  

In [4]:
#Set up Splinter 
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless = False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 100.0.4896
[WDM] - Get LATEST driver version for 100.0.4896
[WDM] - Driver [C:\Users\Terence Lin\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache


In [7]:
nvidia_gpus = ['https://www.zoolert.com/computers/videocards/nvidia/rtx-3070/instockhistory.php',
               'https://www.zoolert.com/computers/videocards/nvidia/rtx-3070-ti/instockhistory.php',
              'https://www.zoolert.com/computers/videocards/nvidia/rtx-3080-ti/instockhistory.php',
               'https://www.zoolert.com/computers/videocards/nvidia/rtx-3090/instockhistory.php',
               'https://www.zoolert.com/computers/videocards/nvidia/rtx-3080/instockhistory.php',
               'https://www.zoolert.com/computers/videocards/nvidia/rtx-3060ti/instockhistory.php',
               'https://www.zoolert.com/computers/videocards/nvidia/rtx-3060/instockhistory.php'
              ]
models = ['rtx-3070','rtx-3070ti','rtx-3080ti','rtx-3090','rtx-3080','rtx-3060ti','rtx-3060']
chipset = []
tags = []
descriptions = []
for values,references in enumerate(nvidia_gpus):
    browser.visit(references)
    browser.find_by_name('history_length').first.select('-1')

for values,references in enumerate(nvidia_gpus):
    browser.visit(references)
    html = browser.html
    gpu_soup = soup(html,'html.parser')
    data = gpu_soup.find_all('tr',role = "row")
    tags.append(data)

    for value,link in enumerate(data):
        descriptions.append(link.get_text(separator = "\n"))
        if( 'T' not in descriptions[value].strip().split("\n")[-1] ):  
            chipset.append(models[values])

In [8]:
temp = []
time = []
chip = []
stores = []
for values, links in enumerate(tags):
    for refs in links:
        temp.append(refs.get_text(separator = "\n"))
        
for times, data in enumerate(temp):
    if( 'T' not in temp[times].strip().split("\n")[-1] ):  
        time.append(temp[times].strip().split("\n")[-1])
        stores.append(temp[times].strip().split("\n")[0])
company = ["Nvidia"] * len(stores)
print(len(stores))
print(len(time))
print(len(company))
while(len(chipset)!=len(time)):
    chipset.append("rtx-3060")

2017
2017
2017


In [9]:
final_conversion = []
time = [value for value in time if value != "Total Time"]
for timestamps, stamps in enumerate(time):
        
    total_time = 0
    cleaned_time = [int(ele) for ele in re.split('[a-z]',stamps) if ele.strip()]
    if len(cleaned_time) is 3: 
        scalar =1440
        total_time = scalar * cleaned_time[0]
        scalar = 60
        total_time = total_time +(scalar*cleaned_time[1])
        scalar = 1
        total_time = total_time + (scalar*cleaned_time[2])
    elif len(cleaned_time) is 2: 
        scalar = 60
        total_time = total_time +(scalar*cleaned_time[0])
        scalar = 1
        total_time = total_time + (scalar*cleaned_time[1])
    else:
        total_time = cleaned_time[0]
    final_conversion.append(total_time)
print(final_conversion)

[2104, 288, 1924, 216, 7100, 5264, 2440, 4, 12448, 176, 12, 272, 4, 16, 8, 1, 1, 16, 40, 1188, 40, 4, 548, 500, 1, 88, 64, 4, 580, 88, 12, 1, 9868, 40, 28, 344, 1069, 1, 336, 1, 4, 1188, 4, 1, 1, 4, 1, 1, 1, 1, 18824, 4, 280, 656, 852, 1584, 2820, 476, 1, 1896, 1, 24, 4, 4, 8, 72, 100, 308, 172, 1, 60, 8, 8, 8, 216, 4, 4, 1, 1, 4, 1, 1, 1, 1, 1, 4, 1, 4, 1, 1, 4, 1, 4, 1, 4, 4, 1, 28, 4, 1, 4, 4, 4, 1, 4, 8, 8, 1, 1, 1, 4, 4, 1, 28, 28, 212, 52, 1, 32, 8, 1, 1, 1, 28, 1404, 1913, 1, 1, 24, 1, 1, 1, 1, 1, 8, 24, 1, 1, 1, 3, 4, 3, 12, 1, 3, 24, 1, 1, 3, 1, 1, 1, 27, 27, 27, 24, 3, 24, 1, 1, 1, 1, 24, 3, 27, 1, 2, 2, 12, 2, 4, 2, 5, 2, 4, 1, 28, 4, 12, 8, 9, 2, 3, 28, 28, 28, 28, 118, 12, 28, 1209, 4, 4, 1, 29, 4, 51, 2, 3, 14, 3, 2, 1, 3, 2, 32, 1, 1, 2, 1, 1, 1, 1, 3, 1, 1, 2, 1, 4, 2, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 2, 2, 2, 1, 1, 2, 1, 12, 1, 1, 4, 4, 1, 1, 1, 1, 1, 7, 36, 3, 180, 4, 57, 375, 32, 89, 274, 7040, 54, 66, 1, 1, 1, 85, 27, 2180, 127, 7, 52, 27, 159, 1, 35, 1, 187, 133, 52, 

In [10]:
df= pd.DataFrame({
    'stores':stores,
    'Company':company,
    'chipset':chipset,
    'time':final_conversion})
df

,stores,Company,chipset,time
0,Amazon,Nvidia,rtx-3070,2104
1,Amazon,Nvidia,rtx-3070,288
2,Amazon,Nvidia,rtx-3070,1924
3,Amazon,Nvidia,rtx-3070,216
4,Amazon,Nvidia,rtx-3070,7100
...,...,...,...,...
2012,Amazon,Nvidia,rtx-3060,152
2013,Amazon,Nvidia,rtx-3060,148
2014,Amazon,Nvidia,rtx-3060,95
2015,Amazon,Nvidia,rtx-3060,505


In [11]:
filepath = Path('CSV/stock_and_time-Nvidia.csv')
df.to_csv(filepath)

In [14]:
amd_gpus = ['https://www.zoolert.com/computers/videocards/radeon/rx5700/instockhistory.php',
           'https://www.zoolert.com/computers/videocards/radeon/6600xt/instockhistory.php',
           'https://www.zoolert.com/computers/videocards/radeon/rx6600/instockhistory.php',
           'https://www.zoolert.com/computers/videocards/radeon/rx6700xt/instockhistory.php',
           'https://www.zoolert.com/computers/videocards/radeon/rx6800xt/instockhistory.php',
           'https://www.zoolert.com/computers/videocards/radeon/rx6900xt/instockhistory.php'
           ]

models = ['rx5700','600xt','rx6600','rx6700xt','rx6800xt','rx6900xt']
amd_tags = []
amd_chipset = []
amd_descriptions = []
for values,references in enumerate(amd_gpus):
    browser.visit(references)
    html = browser.html
    gpu_soup = soup(html,'html.parser')
    browser.find_by_name('history_length').first.select('-1')

for values,references in enumerate(amd_gpus):
    browser.visit(references)
    html = browser.html
    gpu_soup = soup(html,'html.parser')
    data = gpu_soup.find_all('tr',role = "row")
    amd_tags.append(data)

    for value,link in enumerate(data):
        amd_descriptions.append(link.get_text(separator = "\n"))
        if( 'T' not in amd_descriptions[value].strip().split("\n")[-1] ):  
            amd_chipset.append(models[values])

In [26]:
amd_temp = []
amd_time = []
amd_stores = []
for values, links in enumerate(amd_tags):
    for refs in links:
        amd_temp.append(refs.get_text(separator = "\n"))
        
for times, data in enumerate(amd_temp):
    if( 'T' not in amd_temp[times].strip().split("\n")[-1] ):  
        amd_time.append(amd_temp[times].strip().split("\n")[-1])
        amd_stores.append(amd_temp[times].strip().split("\n")[0])
print(len(amd_stores))
print(len(amd_time))
print(len(amd_chipset))

2269
2269
2269


In [28]:
amd_final_conversion = []
amd_time = [value for value in amd_time if value != "Total Time"]
for timestamps, stamps in enumerate(amd_time):
        
    total_time = 0
    cleaned_time = [int(ele) for ele in re.split('[a-z]',stamps) if ele.strip()]
    if len(cleaned_time) is 3: 
        scalar =1440
        total_time = scalar * cleaned_time[0]
        scalar = 60
        total_time = total_time +(scalar*cleaned_time[1])
        scalar = 1
        total_time = total_time + (scalar*cleaned_time[2])
    elif len(cleaned_time) is 2: 
        scalar = 60
        total_time = total_time +(scalar*cleaned_time[0])
        scalar = 1
        total_time = total_time + (scalar*cleaned_time[1])
    else:
        total_time = cleaned_time[0]
    amd_final_conversion.append(total_time)
print(amd_final_conversion)

[1, 1, 180, 180, 1, 180, 1, 1, 1, 1, 180, 1, 1, 1, 1, 1, 180, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 182, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 360, 1, 1, 1, 1, 1, 1, 1, 1, 360, 1, 1, 1, 1, 1, 1, 360, 360, 1, 60, 1, 1, 1, 1, 360, 1, 1, 1, 1, 1800, 1, 1, 360, 1, 1, 1, 1, 1, 1, 60, 360, 1, 1, 1, 1, 1, 360, 360, 360, 360, 360, 1, 360, 1800, 5040, 60, 360, 11160, 3240, 1, 60, 3240, 360, 540, 3240, 1, 27720, 360, 3240, 1, 3600, 1, 1, 1, 6840, 1, 1, 60, 1, 3240, 1, 1, 1, 1, 1, 360, 1, 1, 3240, 1, 1, 1, 1, 33840, 360, 8280, 37080, 1800, 3240, 1800, 1, 16200, 1, 360, 360, 360, 1, 60, 1, 60, 360, 360, 1, 360, 60, 360, 360, 1, 360, 360, 360, 60, 1, 360, 1, 1, 60, 1, 60, 60, 1, 40680, 1, 1, 1, 1, 1, 1, 55440, 1, 360, 1, 1, 360, 1, 3240, 47160, 1, 1, 1, 360, 1, 360, 132840, 360, 7920, 60, 360, 1, 360, 7920, 360, 1, 60, 1, 30960, 21205, 1, 3565, 325, 13645, 1045, 360, 1, 1, 1, 180, 60, 5400, 360, 1, 1, 60, 1, 360, 480, 1, 3240, 1, 1, 1800, 1800, 60, 1800, 3240, 1800, 1, 1, 3240, 

In [29]:
amd_df= pd.DataFrame({
    'stores':amd_stores,
    'chipset':amd_chipset,
    'time':amd_final_conversion})
amd_df

,stores,chipset,time
0,NewEgg,rx5700,1
1,NewEgg,rx5700,1
2,NewEgg,rx5700,180
3,NewEgg,rx5700,180
4,NewEgg,rx5700,1
...,...,...,...
2264,Amazon,rx6900xt,4
2265,Amazon,rx6900xt,6
2266,Amazon,rx6900xt,1
2267,Amazon,rx6900xt,1


In [31]:
filepath = Path('CSV/stock_and_time-AMD.csv')
amd_df.to_csv(filepath)